In [1]:
import pandas as pd
import numpy as np
from functools import reduce

AlloDocPHCS        = pd.read_csv('data/health/allo-doc-PHCS_2017.csv', encoding='utf-8')
DisSubdisDoc       = pd.read_csv('data/health/dis-subdis-doctors_2017.csv', encoding='utf-8')
PhysicianCHCS      = pd.read_csv('data/health/physician-CHCS_2017.csv', encoding='utf-8')
SurgeonCHCS        = pd.read_csv('data/health/surgeons-CHCS_2017.csv', encoding='utf-8')
WorkerMaleSubCen   = pd.read_csv('data/health/worker-male-subcen_2017.csv', encoding='utf-8')
WorkerFemaleSubCen = pd.read_csv('data/health/worker-female-subcen_2017.csv', encoding='utf-8')
AssisMalePHCS      = pd.read_csv('data/health/assistant-male-PHCS_2017.csv', encoding='utf-8')
AssisFemalePHCS    = pd.read_csv('data/health/assistant-female-PHCS_2017.csv', encoding='utf-8')
NurseStaffPHCCHC   = pd.read_csv('data/health/nursing-staff-PHCS-CHCS_2017.csv', encoding='utf-8')
FacilitiesCHCS     = pd.read_csv('data/health/facilities-CHCS_2017.csv', encoding='utf-8')
FacilitiesPHCS     = pd.read_csv('data/health/facilities-PHCS_2017.csv', encoding='utf-8')
RadiographerCHCS   = pd.read_csv('data/health/radiographers-CHCS_2017.csv', encoding='utf-8')
VillageCovered     = pd.read_csv('data/health/villg-coveredby-centre_2017.csv', encoding='utf-8')
AreaCovered        = pd.read_csv('data/health/rural-area-covered-centre_2017.csv', encoding='utf-8')
InfantMorRate      = pd.read_csv('data/health/infant-mortality-rate_2017.csv', encoding='utf-8')
RuralPopCovered    = pd.read_csv('data/health/rural-population-centre_2017.csv', encoding='utf-8')
PharmacistPHCCHC   = pd.read_csv('data/health/pharmacists-PHCS-CHCS_2017.csv', encoding='utf-8')
FunctioningPHCS    = pd.read_csv('data/health/functioning-PHCS_2017.csv', encoding='utf-8')
#FunctioningPHCS    = pd.read_csv('data/health/year-wise-PHCS.csv', encoding='utf-8')
#FunctioningCHCS    = pd.read_csv('data/health/year-wise-CHCS.csv', encoding='utf-8')
#FunctioningSubcen  = pd.read_csv('data/health/year-wise-Subcentre.csv', encoding='utf-8')

# Health data methods

In [44]:
def getHealthCols(df,colheader):
    df.columns = df.columns.str.replace('Number of Primary Health Centres - ', '')
    df = df.add_prefix(colheader)
    cols = list(df.columns.values)[:2]
    df = df.rename(index=str, columns={cols[0]:'S. No.',cols[1]:'State'})
    return df

def getFacilitiesCols(df,colheader):
    df.columns = df.columns.str.replace('Number of Community Health Centres - ', '')
    df.columns = df.columns.str.replace('No. of CHC having a regular supply of - ', '')
    df = df.add_prefix(colheader)
    cols = list(df.columns.values)[:2]
    df = df.rename(index=str, columns={cols[0]:'State'})
    return df


geography  = pd.read_csv('data/geography.csv',encoding='UTF-8')
geo = geography[['geo_level','geo_code','name']]

# Calculating country data
def setCountryData(df,cols):
    for x in cols:
         df.loc[df['geo_level'] == 'country', x]= df.loc[df['geo_level'] == 'state', x].sum()
    return df;

def getHealthGeo(df,varname):
    df = df.rename(index=str, columns={'State':'name'})
    dfgeo = geo.merge(df ,on=['name'],how='left')
    cols = list(df.columns.values)[1:]
    dfgeoCountry = setCountryData(dfgeo,cols)
    dfgeoCountry = dfgeoCountry.fillna(0)
    dfgeoFinal = pd.melt(dfgeoCountry, id_vars=['geo_level', 'geo_code','name'], var_name=varname, value_name="total")
    dfgeoFinal = dfgeoFinal.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1)
    dfgeoFinal['total'] = dfgeoFinal['total'].astype(int)
    return dfgeoFinal;

# Combined health data

In [45]:
AlloDocPHCSf       = getHealthCols(AlloDocPHCS,'Allopathic_doctor_')
DisSubdisDoc       = DisSubdisDoc.rename(index=str, columns={'State/ UT':'State'})
PhysicianCHCSf     = getHealthCols(PhysicianCHCS,'Physician_CHCS_')
SurgeonCHCSf       = getHealthCols(SurgeonCHCS,'Surgeon_CHCS_')
WorkerMaleSubCenf  = getHealthCols(WorkerMaleSubCen,'Workers_male_subcentre_')
WorkerFemaleSubCenf= getHealthCols(WorkerFemaleSubCen,'Workers_female_subcentre_')
AssisMalePHCSf     = getHealthCols(AssisMalePHCS,'Assistant_male_PHCS_')
AssisFemalePHCSf   = getHealthCols(AssisFemalePHCS,'Assistant_female_PHCS_')
NurseStaffPHCCHCf  = getHealthCols(NurseStaffPHCCHC,'Nurse_PHCS_CHCS_')
FacilitiesCHCSf    = getFacilitiesCols(FacilitiesCHCS,'CHCS_facilities_')
FacilitiesPHCSf    = getHealthCols(FacilitiesPHCS,'PHCS_facilities_')
RadiographerCHCSf  = getHealthCols(RadiographerCHCS,'Radiographer_CHCS_')
VillageCovered     = VillageCovered.rename(index=str, columns={'State/ UT':'State'})
AreaCovered        = AreaCovered.rename(index=str, columns={'State/ UT':'State'})
InfantMorRate      = InfantMorRate.rename(index=str, columns={'State/ UT':'State'})
RuralPopCovered    = RuralPopCovered.rename(index=str, columns={'State/ UT':'State'})
PharmacistPHCCHCf  = getHealthCols(PharmacistPHCCHC,'Pharmacist_PHCS_CHCS_')
FunctioningPHCS    = FunctioningPHCS.rename(index=str, columns={'State/ UT':'State'})

# Combining all the df
healthdf    =  [AlloDocPHCSf,DisSubdisDoc,PhysicianCHCSf,SurgeonCHCSf,WorkerMaleSubCenf,WorkerFemaleSubCenf,AssisMalePHCSf,AssisFemalePHCSf,NurseStaffPHCCHCf,FacilitiesCHCSf,FacilitiesPHCSf,RadiographerCHCSf,VillageCovered,AreaCovered,InfantMorRate,RuralPopCovered,PharmacistPHCCHCf,FunctioningPHCS]
healthdf2017 = reduce(lambda left,right: pd.merge(left,right, on=['State']),healthdf)
healthdf2017.replace('A & N Island','ANDAMAN & NICOBAR ISLANDS',inplace=True)
healthdf2017.replace('Delhi','NCT OF DELHI',inplace=True)
healthdf2017['State'] = healthdf2017['State'].str.upper()
healthdf2017.insert(2,'Year','2017')
healthdf2017.head()

,S. No._x,State,Year,Allopathic_doctor_Required - [R],Allopathic_doctor_Sanctioned - [S],Allopathic_doctor_In Position - [P],Allopathic_doctor_Vacant - [S-P],Allopathic_doctor_Shortfall - [R-P],Allopathic_doctor_Note of State/ UT,Allopathic_doctor_Note of Vacant - [S-P],...,Total PHCs functioning,Number of PHCs functioning - with 4+ - doctors,Number of PHCs functioning - with 3 - doctors,Number of PHCs functioning - with 2 - doctors,Number of PHCs functioning - with 1 - doctor,Number of PHCs functioning - without - doctor,Number of PHCs functioning - without - lab tech.,Number of PHCs functioning - without - pharma.,Number of PHCs functioning - with - lady - doctor,Note of State/ UT_y
0,1,ANDHRA PRADESH,2017,1147,1952.0,1644,308.0,NaN,NaN,NaN,...,1147,11,31,476,629,0,364,329,518,NaN
1,2,ARUNACHAL PRADESH,2017,143,NaN,122,NaN,21.0,NaN,NaN,...,143,3,6,27,81,40,55,55,36,NaN
2,3,ASSAM,2017,1014,NaN,1048,NaN,NaN,NaN,NaN,...,1014,67,63,182,626,76,87,110,183,NaN
3,4,BIHAR,2017,1899,2078.0,1786,292.0,113.0,#,NaN,...,1899,439,41,56,1363,0,256,201,156,NaN
4,5,CHHATTISGARH,2017,785,798.0,341,457.0,444.0,NaN,NaN,...,785,0,6,71,318,390,303,185,66,NaN


# Villages covered by centres

In [68]:
villcovscols = ['State','Average Number of Villages covered by a sub Centre','Average Number of Villages covered by a PHC','Average Number of Villages covered by a CHC']

villcovs2017 = healthdf2017[villcovscols]
villcovsGeo2017 = getHealthGeo(villcovs2017,'vilagescovtemp')
villcovsGeo2017.insert(3,'villagescovered',villcovsGeo2017['vilagescovtemp'].map(lambda x: 'Sub Centre' if 'sub Centre' in x else ('PHCS' if 'PHC' in x else 'CHCS')))
villcovsGeo2017.insert(2,'geo_version','2011')
villcovsGeo2017.insert(5,'year','2017')
villcovsGeo2017 =  villcovsGeo2017.drop(['vilagescovtemp'],axis=1)
villcovsGeo2017

# getting default year data
villcovsdefault =  villcovsGeo2017.copy(deep=True)
villcovsdefault['total'] = 0
villcovsdefault['year'] = '2017'
villcovsdefault

#villcovsGeo2017.to_csv('data/villagescovered_2017.csv', index=False)
#villcovsdefault.to_csv('data/villagescovered_default.csv', index=False)
villcovsGeo2017.head()

,geo_level,geo_code,geo_version,villagescovered,year,total
0,country,IN,2011,Sub Centre,2017,165
1,country,IN,2011,PHCS,2017,888
2,country,IN,2011,CHCS,2017,3859
3,district,532,2011,Sub Centre,2017,0
4,district,532,2011,PHCS,2017,0


# Rural population covered by centres

In [70]:
ruralpopcols = ['State','Average Rural Population [Census 2011] covered by a Sub Centre','Average Rural Population [Census 2011] covered by a PHC','Average Rural Population [Census 2011] covered by a CHC']

ruralpopcov2017    = healthdf2017[ruralpopcols]
ruralpopcovGeo2017 = getHealthGeo(ruralpopcov2017,'ruralpopcovtemp')
ruralpopcovGeo2017.insert(3,'ruralpopcovered',ruralpopcovGeo2017['ruralpopcovtemp'].map(lambda x: 'Sub Centre' if 'Sub Centre' in x else ('PHCS' if 'PHC' in x else 'CHCS')))
ruralpopcovGeo2017.insert(2,'geo_version','2011')
ruralpopcovGeo2017.insert(5,'year','2017')
ruralpopcovGeo2017 =  ruralpopcovGeo2017.drop(['ruralpopcovtemp'],axis=1)
ruralpopcovGeo2017

# getting default year data
ruralpopcovdefault =  ruralpopcovGeo2017.copy(deep=True)
ruralpopcovdefault['total'] = 0
ruralpopcovdefault['year'] = '2017'
ruralpopcovdefault

#ruralpopcovGeo2017.to_csv('data/ruralpopcovered_2017.csv', index=False)
#ruralpopcovdefault.to_csv('data/ruralpopcovered_default.csv', index=False)
ruralpopcovGeo2017.head()

,geo_level,geo_code,geo_version,ruralpopcovered,year,total
0,country,IN,2011,Sub Centre,2017,189274
1,country,IN,2011,PHCS,2017,1012714
2,country,IN,2011,CHCS,2017,4446337
3,district,532,2011,Sub Centre,2017,0
4,district,532,2011,PHCS,2017,0


# Nursing staff at PHCS and CHCS

In [72]:
NursePHCCHCcols   = ['State','Nurse_PHCS_CHCS_Required - [R]','Nurse_PHCS_CHCS_In Position - [P]']

NursePHCCHC2017    = healthdf2017[NursePHCCHCcols]
NursePHCCHCGeo2017 = getHealthGeo(NursePHCCHC2017,'nursephcchctemp')
NursePHCCHCGeo2017.insert(3,'nursestaffphcschcs',NursePHCCHCGeo2017['nursephcchctemp'].map(lambda x: 'Required' if 'Required' in x else 'Available'))
NursePHCCHCGeo2017.insert(2,'geo_version','2011')
NursePHCCHCGeo2017.insert(5,'year','2017')
NursePHCCHCGeo2017 =  NursePHCCHCGeo2017.drop(['nursephcchctemp'],axis=1)
NursePHCCHCGeo2017

# getting default year data
NursePHCCHCdefault =  NursePHCCHCGeo2017.copy(deep=True)
NursePHCCHCdefault['total'] = 0
NursePHCCHCdefault['year'] = '2017'
NursePHCCHCdefault

#NursePHCCHCGeo2017.to_csv('data/nursestaffphcschcs_2017.csv', index=False)
#NursePHCCHCdefault.to_csv('data/nursestaffphcschcs_default.csv', index=False)
NursePHCCHCGeo2017.head()

,geo_level,geo_code,geo_version,nursestaffphcschcs,year,total
0,country,IN,2011,Required,2017,65018
1,country,IN,2011,Available,2017,70738
2,district,532,2011,Required,2017,0
3,district,532,2011,Available,2017,0
4,district,146,2011,Required,2017,0


# Allopathic doctors in PHCs

In [74]:
Allodocphcscols = ['State','Allopathic_doctor_Required - [R]','Allopathic_doctor_In Position - [P]']

Allopdoc2017    = healthdf2017[Allodocphcscols]
AllopdocphcGeo2017 = getHealthGeo(Allopdoc2017,'allopdoctemp')
AllopdocphcGeo2017.insert(3,'allopathicdocphcs',AllopdocphcGeo2017['allopdoctemp'].map(lambda x: 'Required' if 'Required' in x else 'Available'))
AllopdocphcGeo2017.insert(2,'geo_version','2011')
AllopdocphcGeo2017.insert(5,'year','2017')
AllopdocphcGeo2017 =  AllopdocphcGeo2017.drop(['allopdoctemp'],axis=1)
AllopdocphcGeo2017

# getting default year data
Allopdocphcdefault =  AllopdocphcGeo2017.copy(deep=True)
Allopdocphcdefault['total'] = 0
Allopdocphcdefault['year'] = '2017'
Allopdocphcdefault

#AllopdocphcGeo2017.to_csv('data/allopathicdocphcs_2017.csv', index=False)
#Allopdocphcdefault.to_csv('data/allopathicdocphcs_default.csv', index=False)
AllopdocphcGeo2017.head()

,geo_level,geo_code,geo_version,allopathicdocphcs,year,total
0,country,IN,2011,Required,2017,25650
1,country,IN,2011,Available,2017,27124
2,district,532,2011,Required,2017,0
3,district,532,2011,Available,2017,0
4,district,146,2011,Required,2017,0


# Dis and sub-dis doctors

In [76]:
dissubdisdoccols = ['State','District Hospital - In Position','Sub District/ Sub Divisional Hospital - In Position']

dissubdisdoc2017    = healthdf2017[dissubdisdoccols]
dissubdisdocGeo2017 = getHealthGeo(dissubdisdoc2017,'dissubdoctemp')
dissubdisdocGeo2017.insert(3,'doctorsdissubhospital',dissubdisdocGeo2017['dissubdoctemp'].map(lambda x: 'District Hospital' if 'District Hospital' in x else 'Sub District Hospital'))
dissubdisdocGeo2017.insert(2,'geo_version','2011')
dissubdisdocGeo2017.insert(5,'year','2017')
dissubdisdocGeo2017 =  dissubdisdocGeo2017.drop(['dissubdoctemp'],axis=1)
dissubdisdocGeo2017

# getting default year data
dissubdisdocdefault =  dissubdisdocGeo2017.copy(deep=True)
dissubdisdocdefault['total'] = 0
dissubdisdocdefault['year'] = '2017'

#dissubdisdocGeo2017.to_csv('data/doctorsdissubhospital_2017.csv', index=False)
#dissubdisdocdefault.to_csv('data/doctorsdissubhospital_default.csv', index=False)
dissubdisdocGeo2017.head()

,geo_level,geo_code,geo_version,doctorsdissubhospital,year,total
0,country,IN,2011,District Hospital,2017,20837
1,country,IN,2011,Sub District Hospital,2017,13353
2,district,532,2011,District Hospital,2017,0
3,district,532,2011,Sub District Hospital,2017,0
4,district,146,2011,District Hospital,2017,0


# Physicians in CHCs

In [78]:
physicianchcscols = ['State','Physician_CHCS_Required - [R]','Physician_CHCS_In Position - [P]']

physicianchcs2017    = healthdf2017[physicianchcscols]
physicianchcsGeo2017 = getHealthGeo(physicianchcs2017 ,'physicianchcstemp')
physicianchcsGeo2017.insert(3,'physicianchcs',physicianchcsGeo2017['physicianchcstemp'].map(lambda x: 'Required' if 'Required' in x else 'Available'))
physicianchcsGeo2017.insert(2,'geo_version','2011')
physicianchcsGeo2017.insert(5,'year','2017')
physicianchcsGeo2017 =  physicianchcsGeo2017.drop(['physicianchcstemp'],axis=1)

# getting default year data
physicianchcsdefault =  physicianchcsGeo2017.copy(deep=True)
physicianchcsdefault['total'] = 0
physicianchcsdefault['year'] = '2017'

#physicianchcsGeo2017.to_csv('data/physicianchcs_2017.csv', index=False)
#physicianchcsdefault.to_csv('data/physicianchcs_default.csv', index=False)
physicianchcsGeo2017.head()

,geo_level,geo_code,geo_version,physicianchcs,year,total
0,country,IN,2011,Required,2017,5624
1,country,IN,2011,Available,2017,864
2,district,532,2011,Required,2017,0
3,district,532,2011,Available,2017,0
4,district,146,2011,Required,2017,0


# Surgeons in CHCS

In [80]:
surgeonchcscols = ['State','Surgeon_CHCS_Required - [R]','Surgeon_CHCS_In Position - [P]']

surgeonchcs2017    = healthdf2017[surgeonchcscols]
surgeonchcsGeo2017 = getHealthGeo(surgeonchcs2017 ,'surgeonchcstemp')
surgeonchcsGeo2017.insert(3,'surgeonchcs',surgeonchcsGeo2017['surgeonchcstemp'].map(lambda x: 'Required' if 'Required' in x else 'Available'))
surgeonchcsGeo2017.insert(2,'geo_version','2011')
surgeonchcsGeo2017.insert(5,'year','2017')
surgeonchcsGeo2017 =  surgeonchcsGeo2017.drop(['surgeonchcstemp'],axis=1)
surgeonchcsGeo2017

# getting default year data
surgeonchcsdefault =  surgeonchcsGeo2017.copy(deep=True)
surgeonchcsdefault['total'] = 0
surgeonchcsdefault['year'] = '2017'
surgeonchcsdefault

#surgeonchcsGeo2017.to_csv('data/surgeonchcs_2017.csv', index=False)
#surgeonchcsdefault.to_csv('data/surgeonchcs_default.csv', index=False)
surgeonchcsGeo2017.head()

,geo_level,geo_code,geo_version,surgeonchcs,year,total
0,country,IN,2011,Required,2017,5624
1,country,IN,2011,Available,2017,758
2,district,532,2011,Required,2017,0
3,district,532,2011,Available,2017,0
4,district,146,2011,Required,2017,0


# Radiographer in CHCS

In [82]:
radiogrchcscols = ['State','Radiographer_CHCS_Required - [R]','Radiographer_CHCS_In Position - [P]']

radiogrchcs2017    = healthdf2017[radiogrchcscols]
radiogrchcsGeo2017 = getHealthGeo(radiogrchcs2017 ,'radiogrchcstemp')
radiogrchcsGeo2017.insert(3,'radiographerchcs',radiogrchcsGeo2017['radiogrchcstemp'].map(lambda x: 'Required' if 'Required' in x else 'Available'))
radiogrchcsGeo2017.insert(2,'geo_version','2011')
radiogrchcsGeo2017.insert(5,'year','2017')
radiogrchcsGeo2017 =  radiogrchcsGeo2017.drop(['radiogrchcstemp'],axis=1)
radiogrchcsGeo2017

# getting default year data
radiogrchcsdefault =  radiogrchcsGeo2017.copy(deep=True)
radiogrchcsdefault['total'] = 0
radiogrchcsdefault['year'] = '2017'
radiogrchcsdefault

#radiogrchcsGeo2017.to_csv('data/radiographerchcs_2017.csv', index=False)
#radiogrchcsdefault.to_csv('data/radiographerchcs_default.csv', index=False)
radiogrchcsGeo2017.head()

,geo_level,geo_code,geo_version,radiographerchcs,year,total
0,country,IN,2011,Required,2017,5624
1,country,IN,2011,Available,2017,2129
2,district,532,2011,Required,2017,0
3,district,532,2011,Available,2017,0
4,district,146,2011,Required,2017,0


# Pharmacist in PHCS and CHCS

In [84]:
pharmaphcchccols = ['State','Pharmacist_PHCS_CHCS_Required - [R]','Pharmacist_PHCS_CHCS_In Position - [P]']

pharmaphcchc2017    = healthdf2017[pharmaphcchccols]
pharmaphcchcGeo2017 = getHealthGeo(pharmaphcchc2017 ,'pharmachcstemp')
pharmaphcchcGeo2017.insert(3,'pharmacistphcschcs',pharmaphcchcGeo2017['pharmachcstemp'].map(lambda x: 'Required' if 'Required' in x else 'Available'))
pharmaphcchcGeo2017.insert(2,'geo_version','2011')
pharmaphcchcGeo2017.insert(5,'year','2017')
pharmaphcchcGeo2017 =  pharmaphcchcGeo2017.drop(['pharmachcstemp'],axis=1)
pharmaphcchcGeo2017

# getting default year data
pharmaphcchcdefault =  pharmaphcchcGeo2017.copy(deep=True)
pharmaphcchcdefault['total'] = 0
pharmaphcchcdefault['year'] = '2017'
pharmaphcchcdefault
pharmaphcchcdefault

#pharmaphcchcGeo2017.to_csv('data/pharmacistphcschcs_2017.csv', index=False)
#pharmaphcchcdefault.to_csv('data/pharmacistphcschcs_default.csv', index=False)
pharmaphcchcGeo2017.head()

,geo_level,geo_code,geo_version,pharmacistphcschcs,year,total
0,country,IN,2011,Required,2017,31274
1,country,IN,2011,Available,2017,25193
2,district,532,2011,Required,2017,0
3,district,532,2011,Available,2017,0
4,district,146,2011,Required,2017,0


# Workers at subcentre

In [86]:
workerssubcencols = ['State','Workers_male_subcentre_Required - [R]','Workers_female_subcentre_Required - [R]','Workers_male_subcentre_In Position - [P]','Workers_female_subcentre_In Position - [P]',]

workerssubcen2017    = healthdf2017[workerssubcencols]
workerssubcenGeo2017 = getHealthGeo(workerssubcen2017 ,'worksubcentemp')
workerssubcenGeo2017.insert(3,'workersubcen',workerssubcenGeo2017['worksubcentemp'].map(lambda x: 'Required' if 'Required' in x else 'Available'))
workerssubcenGeo2017.insert(4,'gender',workerssubcenGeo2017['worksubcentemp'].map(lambda x: 'Female' if 'female' in x else 'male'))
workerssubcenGeo2017.insert(2,'geo_version','2011')
workerssubcenGeo2017.insert(6,'year','2017')
workerssubcenGeo2017 =  workerssubcenGeo2017.drop(['worksubcentemp'],axis=1)

# getting default year data
workerssubcendefault =  workerssubcenGeo2017.copy(deep=True)
workerssubcendefault['total'] = 0
workerssubcendefault['year'] = '2017'
workerssubcendefault

#workerssubcenGeo2017.to_csv('data/workersubcentre_2017.csv', index=False)
#workerssubcendefault.to_csv('data/workersubcentre_default.csv', index=False)
workerssubcenGeo2017.head()

,geo_level,geo_code,geo_version,workersubcen,gender,year,total
0,country,IN,2011,Required,male,2017,156231
1,country,IN,2011,Required,Female,2017,156231
2,country,IN,2011,Available,male,2017,56263
3,country,IN,2011,Available,Female,2017,198356
4,district,532,2011,Required,male,2017,0


# Assistant at PHCS

In [88]:
assisphcscols = ['State','Assistant_male_PHCS_Required - [R]','Assistant_female_PHCS_Required - [R]','Assistant_male_PHCS_In Position - [P]','Assistant_female_PHCS_In Position - [P]']

assisphcs2017    = healthdf2017[assisphcscols]
assisphcsGeo2017 = getHealthGeo(assisphcs2017 ,'assisphcstemp')
assisphcsGeo2017.insert(3,'assistantphcs',assisphcsGeo2017['assisphcstemp'].map(lambda x: 'Required' if 'Required' in x else 'Available'))
assisphcsGeo2017.insert(4,'gender',assisphcsGeo2017['assisphcstemp'].map(lambda x: 'Female' if 'female' in x else 'male'))
assisphcsGeo2017.insert(2,'geo_version','2011')
assisphcsGeo2017.insert(6,'year','2017')
assisphcsGeo2017 =  assisphcsGeo2017.drop(['assisphcstemp'],axis=1)

# getting default year data
assisphcsdefault =  assisphcsGeo2017.copy(deep=True)
assisphcsdefault['total'] = 0
assisphcsdefault['year'] = '2017'

#assisphcsGeo2017.to_csv('data/assistantphcs_2017.csv', index=False)
#assisphcsdefault.to_csv('data/assistantphcs_default.csv', index=False)
assisphcsGeo2017.head()

,geo_level,geo_code,geo_version,assistantphcs,gender,year,total
0,country,IN,2011,Required,male,2017,25650
1,country,IN,2011,Required,Female,2017,25650
2,country,IN,2011,Available,male,2017,12288
3,country,IN,2011,Available,Female,2017,14267
4,district,532,2011,Required,male,2017,0


# Facilities CHCS 

In [90]:
facichcscols = ['State','CHCS_facilities_With at least 30 beds - Col. 10','CHCS_facilities_With functional Laboratory - Col. 5','CHCS_facilities_With functional O.T. - Col. 6','CHCS_facilities_With functional Labor Room - Col. 7','CHCS_facilities_With New Born Care Corner - Col. 9','CHCS_facilities_With functional X-Ray machine - Col. 11','CHCS_facilities_With quarters for specialist Doctors - Col. 12','CHCS_facilities_With referral transport available - Col. 14','CHCS_facilities_Allopathic drugs for common ailments - Col. 17','CHCS_facilities_AYUSH drugs for common ailments - Col. 18']

facichcs2017    = healthdf2017[facichcscols]
facichcs2017    = facichcs2017.rename(index=str, columns={'CHCS_facilities_With at least 30 beds - Col. 10':'Min 30 beds','CHCS_facilities_With functional Laboratory - Col. 5':'Laboratory','CHCS_facilities_With functional O.T. - Col. 6':'O.T','CHCS_facilities_With functional Labor Room - Col. 7':'Labour Room','CHCS_facilities_With New Born Care Corner - Col. 9':'New Born Care Corner','CHCS_facilities_With functional X-Ray machine - Col. 11':'X-ray','CHCS_facilities_With quarters for specialist Doctors - Col. 12':'Specialist Doctors Quarter','CHCS_facilities_With referral transport available - Col. 14':'Referral Transport','CHCS_facilities_Allopathic drugs for common ailments - Col. 17':'Allopatic Drugs','CHCS_facilities_AYUSH drugs for common ailments - Col. 18':'Ayush Drugs'})
facichcsGeo2017 = getHealthGeo(facichcs2017 ,'facilitieschcs')
facichcsGeo2017.insert(2,'geo_version','2011')
facichcsGeo2017.insert(4,'year','2017')

# getting default year data
facichcsdefault =  facichcsGeo2017.copy(deep=True)
facichcsdefault['total'] = 0
facichcsdefault['year'] = '2017'

#facichcsGeo2017.to_csv('data/facilitieschcs_2017.csv', index=False)
#facichcsdefault.to_csv('data/facilitieschcs_default.csv', index=False)
facichcsGeo2017.head()

,geo_level,geo_code,geo_version,facilitieschcs,year,total
0,country,IN,2011,Min 30 beds,2017,4083
1,country,IN,2011,Laboratory,2017,5303
2,country,IN,2011,O.T,2017,4696
3,country,IN,2011,Labour Room,2017,5186
4,country,IN,2011,New Born Care Corner,2017,4722


# Facilities PHCS

In [92]:
faciphcscols = ['State','PHCS_facilities_With Labour Room - Number','PHCS_facilities_With OT - Number','PHCS_facilities_With at least 4 beds - Number','PHCS_facilities_With Telephone - Number','PHCS_facilities_With Computer - Number','PHCS_facilities_Referral Transport']

faciphcs2017    = healthdf2017[faciphcscols]
faciphcs2017    = faciphcs2017.rename(index=str, columns={'PHCS_facilities_With Labour Room - Number':'Labour Room','PHCS_facilities_With OT - Number':'OT','PHCS_facilities_With at least 4 beds - Number':'Min 4 Beds','PHCS_facilities_With Telephone - Number':'Telephone','PHCS_facilities_With Computer - Number':'Computer','PHCS_facilities_Referral Transport':'Referral Transport'})
faciphcsGeo2017 = getHealthGeo(faciphcs2017 ,'facilitiesphcs')
faciphcsGeo2017.insert(2,'geo_version','2011')
faciphcsGeo2017.insert(4,'year','2017')
faciphcsGeo2017

# getting default year data
faciphcsdefault =  faciphcsGeo2017.copy(deep=True)
faciphcsdefault['total'] = 0
faciphcsdefault['year'] = '2017'

#faciphcsGeo2017.to_csv('data/facilitiesphcs_2017.csv', index=False)
#faciphcsdefault.to_csv('data/facilitiesphcs_default.csv', index=False)
faciphcsGeo2017.head()

,geo_level,geo_code,geo_version,facilitiesphcs,year,total
0,country,IN,2011,Labour Room,2017,17688
1,country,IN,2011,OT,2017,9422
2,country,IN,2011,Min 4 Beds,2017,19559
3,country,IN,2011,Telephone,2017,13918
4,country,IN,2011,Computer,2017,16688
